In [2]:
import sys
import os
import argparse
import random

SCRIPT_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(SCRIPT_DIR)
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# Third-party
import numpy as np
import torch
import tensorflow as tf
from datasets import load_dataset

# Communication
from basestation import User, EdgeServer
from user_association import uncertainty_aware_offloading
from utils import (
    estimate_workload,
    generate_rayleigh_coeffs,
    bit_size_text,
)
from config import (
    BANDWIDTH,
    TRANSMIT_POWER,
    NOISE_POWER,
    LOCAL_COMPUTE_CAP,
    EDGE_COMPUTE_CAP,
    MAX_COMPUTE_PER_USER,
    SLM,
    LLM,
    K,
)

# LLM
from model.huggingface_model import HuggingfaceModel
from model import get_model

In [3]:
model = get_model(SLM)
SLM

'meta-llama/Llama-3.2-1B-Instruct'

In [4]:
dataset = load_dataset("Muennighoff/babi", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
ds = dataset.shuffle().select(range(40))
for i in range(40):
    passage = ds[i]["passage"]
    question = dataset[i]["question"]
    answer = dataset[i]["answer"]

The bedroom is south of the hallway.
The hallway is south of the garden.


Bill moved to the kitchen.
Jeff went to the bedroom.
Bill travelled to the hallway.
Fred moved to the bathroom.
Jeff moved to the kitchen.
Jeff got the football there.
Bill went back to the kitchen.
Jeff handed the football to Bill.
Bill handed the football to Jeff.
Jeff passed the football to Bill.
Bill grabbed the milk there.
Fred travelled to the hallway.


Sandra moved to the garden.
Mary got the apple.
John travelled to the hallway.
John got the football there.
Sandra moved to the office.
John moved to the office.
John journeyed to the kitchen.
Mary went back to the garden.
Mary picked up the milk.
John dropped the football.
Mary dropped the milk.
Mary dropped the apple.
Mary got the apple.
Mary went back to the kitchen.


Daniel and Sandra went to the bathroom.
John and Sandra went to the garden.
Sandra and John went back to the kitchen.
Mary and Daniel moved to the garden.
Sandra and John journeyed to the

In [ ]:
import re


def is_correct(prediction: str, answer: str) -> bool:
    prediction = prediction.strip().lower()
    answer = answer.strip().lower()
    return re.search(rf"\b{re.escape(answer)}\b", prediction) is not None

In [ ]:
total = 100
correct = 0
for i in range(total):
    passage = dataset[i]["passage"]
    question = dataset[i]["question"]
    answer = dataset[i]["answer"]
    input = (
        f"Instruction: Answer with only one word. No explanation."
        f"Context: {passage.strip()}\n"
        f"Question: {question.strip()}\n"
        f"Answer(only one word):"
    )
    output = model.generate(input)
    prediction = output[len(input) :]
    print(prediction + "\t" + answer + "\n")

    if is_correct(prediction, answer):
        correct += 1

print(f"Accuracy: {correct}/{total} = {correct/total:.2%}")

In [ ]:
model = get_model(LLM)

In [ ]:
model.generate(input)

In [ ]:
total = 100
correct = 0
for i in range(total):
    passage = dataset[i]["passage"]
    question = dataset[i]["question"]
    answer = dataset[i]["answer"]
    input = (
        f"Context: {passage.strip()}\n"
        f"Question: {question.strip()}\n"
        f"Answer(only one word):"
    )
    output = model.generate(input)
    prediction = output[len(input) :].strip()

    if is_correct(prediction, answer):
        correct += 1

print(f"Accuracy: {correct}/{total} = {correct/total:.2%}")